### **TEXT SUMMARIZATION TOOL**

In [ ]:
!pip install polyglot
!pip install pyicu
!pip install morfessor
!pip install pycld2
!pip install Goslate
!pip install sumy
!pip install gensim
!pip install transformers
!pip install sentencepiece
!pip install langdetect
!pip install googletrans==4.0.0-rc1
!pip install google-cloud-translate
!pip install transformers torch sentencepiece

In [ ]:
import re
import nltk
import goslate
import numpy as np
import networkx as nx
import ipywidgets as widgets
from langdetect import detect
from transformers import pipeline
from nltk.corpus import stopwords
from IPython.display import display
from nltk.cluster.util import cosine_distance

nltk.download("punkt")
nltk.download("stopwords")


# Function to generate text summary
def generate_summary(file_content, extract_percent=30, target_language='en'):
    # Detect the language of the input text using langdetect
    detected_language = detect(file_content)

    # Check if the length of the text is above a certain threshold for summarization
    if len(file_content) > 250:
        # Initialize the summarization pipeline with the appropriate model based on the detected language
        if detected_language == 'en':
            model_path = "facebook/bart-large-cnn"
        elif detected_language == 'hi':
            model_path = "sshleifer/distilbart-cnn-12-6"
        elif detected_language == 'bn':  # Change the model for Bengali
            model_path = "bert-base-multilingual-cased"
        else:
            model_path = "facebook/bart-large-cnn"  # Default to English model

        summarizer = pipeline("summarization", model=model_path)

        # Generate the summary
        summary = summarizer(file_content, max_length=150, min_length=30, do_sample=False)[0]["summary_text"]

        # Check if translation is needed
        if detected_language != target_language:
            # Translation logic if needed
            return summary
        return summary
    else:
        return "Input text is too short for summarization."

# User Interaction
print("Text Summarization Tool")

input_choice = widgets.Dropdown(
    options=['text', 'file'],
    value='text',
    description='Input method:'
)
display(input_choice)

text_input = widgets.Textarea(
    value='',
    placeholder='Enter the text for summarization',
    description='Input text:'
)
display(text_input)

file_input = widgets.FileUpload(
    accept='.txt',
    multiple=False,
    description='Upload text file:'
)
display(file_input)

extract_percent_slider = widgets.FloatSlider(
    value=30.0,
    min=1.0,
    max=100.0,
    step=1.0,
    description='Extraction %:'
)
display(extract_percent_slider)

target_language_dropdown = widgets.Dropdown(
    options=['English', 'Hindi', 'Bengali'],
    value='English',
    description='Target Language:'
)
display(target_language_dropdown)

output = widgets.Output()
display(output)

def summarize_button_clicked(b):
    with output:
        output.clear_output()
        input_method = input_choice.value
        extract_percent = extract_percent_slider.value
        target_language = target_language_dropdown.value.lower()

        if input_method == "text":
            file_content = text_input.value
        elif input_method == "file" and 'file_input' in globals() and file_input.value:
            uploaded_file = file_input.value[list(file_input.value.keys())[0]]
            file_content = uploaded_file['content'].decode("utf-8")
        else:
            print("Invalid input method or file not provided.")
            return

        try:
            summary = generate_summary(file_content, extract_percent, target_language=target_language)
            print("Summarized Text:\n", summary)
        except Exception as e:
            print(f"An error occurred: {e}")

summarize_button = widgets.Button(description="Summarize & Download")
summarize_button.on_click(summarize_button_clicked)
display(summarize_button)
